In [1]:
# Re-using the code from previous notebooks
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
# set hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 10

# Initialize the loss function
# In this case, we use CrossEntropyLoss for classification
# Regression problems would use MSELoss
loss_fn = nn.CrossEntropyLoss()

# Initialize the optimizer, here: Stochastic Gradient Descent
# other options: Adam, RMSprop, etc.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [3]:
# loops over our optimization code
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [4]:
# evaluate the model's performance against the test dataset
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305524  [   64/60000]
loss: 2.297900  [ 6464/60000]
loss: 2.292473  [12864/60000]
loss: 2.293695  [19264/60000]
loss: 2.288860  [25664/60000]
loss: 2.281195  [32064/60000]
loss: 2.278321  [38464/60000]
loss: 2.276467  [44864/60000]
loss: 2.273476  [51264/60000]
loss: 2.272420  [57664/60000]
Test Error: 
 Accuracy: 29.7%, Avg loss: 2.269497 

Epoch 2
-------------------------------
loss: 2.271211  [   64/60000]
loss: 2.265894  [ 6464/60000]
loss: 2.266546  [12864/60000]
loss: 2.250803  [19264/60000]
loss: 2.251344  [25664/60000]
loss: 2.244667  [32064/60000]
loss: 2.235769  [38464/60000]
loss: 2.244012  [44864/60000]
loss: 2.229375  [51264/60000]
loss: 2.226827  [57664/60000]
Test Error: 
 Accuracy: 53.4%, Avg loss: 2.222397 

Epoch 3
-------------------------------
loss: 2.222222  [   64/60000]
loss: 2.216267  [ 6464/60000]
loss: 2.227617  [12864/60000]
loss: 2.185579  [19264/60000]
loss: 2.192087  [25664/60000]
loss: 2.187619  [32064/600

In [10]:
torch.save(model, 'model.pth')
model = torch.load('model.pth', weights_only=False)